In [4]:
# Imports
%pip install plotly

import asyncio
import websockets
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import plotly.express as px
import json
from typing import Dict, List, Tuple
import aiohttp
import time

You should consider upgrading via the 'D:\spanda-ft-work\repos\platform-env-fix\spandaai-platform\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Configuration
BASE_URL = "http://localhost:8090/api"
WS_URL = "ws://localhost:8090/api/ws/document_analysis"

In [ ]:
class AdaptiveLearningAnalyzer:
    def __init__(self):
        self.session = aiohttp.ClientSession()
        self.analysis_cache = {}
        
    async def analyze_document_stream(self, content: str, rubric: dict) -> dict:
        """Streams document analysis using WebSocket connection"""
        async with websockets.connect(WS_URL) as websocket:
            await websocket.send(json.dumps({
                "rubric": rubric,
                "pre_analysis": {
                    "degree": "PhD",
                    "name": "Analysis Stream",
                    "topic": "Adaptive Learning",
                    "pre_analyzed_summary": content[:500]
                },
                "feedback": "Detailed analysis needed"
            }))
            
            results = []
            async for message in websocket:
                data = json.loads(message)
                if data["type"] == "result":
                    return data["data"]
                elif data["type"] == "progress":
                    print(f"Progress: {data['data']['percentage']}% - {data['data']['message']}")
                    results.append(data['data'])
            return results

    async def process_content_chunks(self, content: str, chunk_size: int = 1000) -> List[dict]:
        """Process content in chunks for parallel analysis"""
        async with self.session.post(f"{BASE_URL}/chunk-text", 
                                   json={"text": content, "chunk_size": chunk_size, "overlap": 100}) as response:
            chunks = await response.json()
            
        # Process chunks in parallel
        async with self.session.post(f"{BASE_URL}/process-chunks", 
                                   json={
                                       "chunks": [chunk["text"] for chunk in chunks["chunks"]],
                                       "system_prompt": "Analyze for key concepts and learning objectives",
                                       "batch_size": 5
                                   }) as response:
            return await response.json()

    async def generate_adaptive_questions(self, content: str, difficulty_distribution: Dict[str, float]) -> List[dict]:
        """Generate questions with adaptive difficulty based on content analysis"""
        questions = []
        difficulties = ["easy", "medium", "hard"]
        
        for difficulty in difficulties:
            count = int(difficulty_distribution[difficulty] * 10)  # 10 questions per difficulty level
            async with self.session.post(f"{BASE_URL}/questions_generation",
                                       json={
                                           "topic": "Adaptive Learning",
                                           "difficulty": difficulty,
                                           "type_of_question": "multiple_choice",
                                           "no_of_questions": count,
                                           "context": content,
                                           "no_of_options": 4,
                                           "numericality": "mixed",
                                           "few_shot": "true"
                                       }) as response:
                result = await response.json()
                questions.extend(result["questions"])
        
        return questions

    def calculate_learning_path(self, analysis_results: dict, questions: List[dict]) -> Tuple[List[dict], float]:
        """Calculate optimal learning path based on analysis and questions"""
        # Create knowledge graph from analysis
        knowledge_points = {}
        for chunk in analysis_results["processed_chunks"]:
            for point in chunk["key_points"]:
                if point not in knowledge_points:
                    knowledge_points[point] = {
                        "difficulty": np.random.random(),  # Simulate difficulty rating
                        "relevance": np.random.random(),   # Simulate relevance score
                        "questions": []
                    }
        
        # Associate questions with knowledge points
        for question in questions:
            # Simulate matching questions to knowledge points
            matched_point = np.random.choice(list(knowledge_points.keys()))
            knowledge_points[matched_point]["questions"].append(question)
        
        # Calculate optimal path
        sorted_points = sorted(knowledge_points.items(), 
                             key=lambda x: (x[1]["difficulty"], -x[1]["relevance"]))
        
        learning_path = []
        total_difficulty = 0
        
        for point, data in sorted_points:
            path_item = {
                "concept": point,
                "difficulty": data["difficulty"],
                "relevance": data["relevance"],
                "questions": data["questions"]
            }
            learning_path.append(path_item)
            total_difficulty += data["difficulty"]
        
        return learning_path, total_difficulty / len(sorted_points)

    async def visualize_learning_path(self, learning_path: List[dict]):
        """Create interactive visualization of learning path"""
        df = pd.DataFrame([
            {
                "concept": item["concept"][:50] + "...",  # Truncate for display
                "difficulty": item["difficulty"],
                "relevance": item["relevance"],
                "question_count": len(item["questions"])
            }
            for item in learning_path
        ])
        
        fig = px.scatter(df, x="difficulty", y="relevance",
                        size="question_count",
                        hover_data=["concept"],
                        title="Learning Path Visualization")
        
        return fig

    async def analyze_and_create_path(self, content: str):
        """Main function to analyze content and create adaptive learning path"""
        # Initial document analysis
        analysis_result = await self.analyze_document_stream(content, {
            "learning_objectives": {
                "criteria_explanation": "Clear learning objectives",
                "score_explanation": "Score based on clarity and measurability",
                "criteria_output": "Detailed output format"
            }
        })
        
        # Process content in chunks
        chunk_results = await self.process_content_chunks(content)
        
        # Generate adaptive questions
        questions = await self.generate_adaptive_questions(
            content,
            {"easy": 0.3, "medium": 0.4, "hard": 0.3}
        )
        
        # Calculate learning path
        learning_path, avg_difficulty = self.calculate_learning_path(chunk_results, questions)
        
        # Visualize results
        viz = await self.visualize_learning_path(learning_path)
        
        return {
            "analysis": analysis_result,
            "learning_path": learning_path,
            "visualization": viz,
            "average_difficulty": avg_difficulty,
            "question_count": len(questions)
        }

In [ ]:
# Usage Example
async def main():
    analyzer = AdaptiveLearningAnalyzer()
    
    # Sample content
    content = """
    [Your educational content here...]
    """
    
    results = await analyzer.analyze_and_create_path(content)
    
    # Display results
    display(HTML("<h2>Analysis Results</h2>"))
    display(results["visualization"])
    print(f"\nAverage Difficulty: {results['average_difficulty']:.2f}")
    print(f"Total Questions Generated: {results['question_count']}")
    
    for i, path_item in enumerate(results["learning_path"], 1):
        print(f"\nStep {i}: {path_item['concept']}")
        print(f"Difficulty: {path_item['difficulty']:.2f}")
        print(f"Relevance: {path_item['relevance']:.2f}")
        print(f"Questions: {len(path_item['questions'])}")

# Run the analysis
await main()